In [ ]:
from IPython.display import display, HTML
import os
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%run ../nb_core/root_dirs.ipynb
setup_syspath_disentangle(False)
%run ../nb_core/disentangle_imports.ipynb

In [ ]:
# ckpt_dir = "/group/jug/ashesh/training/disentangle/2406/D14-M3-S0-L8/2" 
ckpt_dir = "/group/jug/ashesh/training/disentangle/2405/D14-M3-S0-L8/9" 
assert os.path.exists(ckpt_dir)

In [ ]:
def get_dtype(ckpt_fpath):
    if os.path.isdir(ckpt_fpath):
        ckpt_fpath = ckpt_fpath[:-1] if ckpt_fpath[-1] == '/' else ckpt_fpath
    elif os.path.isfile(ckpt_fpath):
        ckpt_fpath = os.path.dirname(ckpt_fpath)
    assert ckpt_fpath[-1] != '/'
    return int(ckpt_fpath.split('/')[-2].split('-')[0][1:])

In [ ]:
dtype = get_dtype(ckpt_dir)
dtype

In [ ]:
from disentangle.data_loader.ht_iba1_ki67_rawdata_loader import SubDsetType

image_size_for_grid_centers = 64
mmse_count = 50
subdset_type = SubDsetType.OnlyIba1P70


custom_image_size = None
data_t_list = None #[0,1,2,3]

save_comparative_plots =False
enable_calibration = False
batch_size = 64
num_workers = 4
COMPUTE_LOSS = False
use_deterministic_grid = None
threshold = None # 0.02
compute_kl_loss = False
evaluate_train = False# inspect training performance
eval_datasplit_type = DataSplitType.Test 
val_repeat_factor = None
psnr_type = 'range_invariant' #'simple', 'range_invariant'

In [ ]:
%run ../nb_core/config_loader.ipynb

In [ ]:
tokens = ckpt_dir.split('/')
idx = tokens.index('disentangle')
# if config.model.model_type == 25 and tokens[idx+1] == '2312':
#     config.model.model_type = ModelType.LadderVAERestrictedReconstruction

In [ ]:
from disentangle.core.sampler_type import SamplerType
from disentangle.core.loss_type import LossType
from disentangle.data_loader.ht_iba1_ki67_rawdata_loader import SubDsetType
# from disentangle.core.lowres_merge_type import LowresMergeType


with config.unlocked():
    if dtype == DataType.HTIba1Ki67:
        config.data.subdset_type = subdset_type
        config.data.empty_patch_replacement_enabled = False



    # older config updates.
    config.model.skip_nboundary_pixels_from_loss = None
    if config.model.model_type == ModelType.UNet and 'n_levels' not in config.model:
        config.model.n_levels = 4
    if config.data.sampler_type == SamplerType.NeighborSampler:
        config.data.sampler_type = SamplerType.DefaultSampler
        config.loss.loss_type = LossType.Elbo
        config.data.grid_size = config.data.image_size
    if 'ch1_fpath_list' in config.data:
        config.data.ch1_fpath_list = config.data.ch1_fpath_list[:1]
        config.data.mix_fpath_list = config.data.mix_fpath_list[:1]
    if config.data.data_type == DataType.Pavia2VanillaSplitting:
        if 'channel_2_downscale_factor' not in config.data:
            config.data.channel_2_downscale_factor = 1
    if config.model.model_type == ModelType.UNet and 'init_channel_count' not in config.model:
        config.model.init_channel_count = 64
    
    if 'skip_receptive_field_loss_tokens' not in config.loss:
        config.loss.skip_receptive_field_loss_tokens = []
    
    
    
    if 'lowres_merge_type' not in config.model.encoder:
        config.model.encoder.lowres_merge_type = 0
    if 'validtarget_random_fraction' in config.data:
        config.data.validtarget_random_fraction = None
    
    if config.data.data_type == DataType.TwoDset:
        config.model.model_type = ModelType.LadderVae
        for key in config.data.dset1:
            config.data[key] = config.data.dset1[key]
    
    if 'dump_kth_frame_prediction' in config.training:
        config.training.dump_kth_frame_prediction = None

    if 'input_is_sum' not in config.data:
        config.data.input_is_sum = False

In [ ]:
# config.data.channel_1 = 0 
# config.data.channel_2 = 3

In [ ]:
dtype = config.data.data_type
assert dtype == DataType.HTIba1Ki67
data_dir = f'{DATA_ROOT}/Stefania/20230327_Ki67_and_Iba1_trainingdata/'

In [ ]:
config.model.noise_model_ch1_fpath = config.model.noise_model_ch1_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')
config.model.noise_model_ch2_fpath = config.model.noise_model_ch2_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')

In [ ]:
%run ../nb_core/disentangle_setup.ipynb

In [ ]:
if config.data.multiscale_lowres_count is not None and custom_image_size is not None:
    model.reset_for_different_output_size(custom_image_size)

In [ ]:
idx = np.random.randint(len(val_dset))
inp_tmp, tar_tmp, *_ = val_dset[idx]
ncols = len(tar_tmp)
nrows = 2
_,ax = plt.subplots(figsize=(4*ncols,4*nrows),ncols=ncols,nrows=nrows)
for i in range(min(ncols,len(inp_tmp))):
    ax[0,i].imshow(inp_tmp[i])

for channel_id in range(ncols):
    ax[1,channel_id].imshow(tar_tmp[channel_id])

In [ ]:
if data_t_list is not None:
    val_dset.reduce_data(t_list=data_t_list)

In [ ]:
highsnr_val_dset = None



In [ ]:
def get_full_input_frame(idx, dset):
    img_tuples, noise_tuples = dset._load_img(idx)
    if len(noise_tuples) > 0:
        factor = np.sqrt(2) if dset._input_is_sum else 1.0
        img_tuples = [x + noise_tuples[0] * factor for x in img_tuples]

    inp = 0
    for nch in img_tuples:
        inp += nch/len(img_tuples)
    h_start, w_start = dset._get_deterministic_hw(idx)
    return inp, h_start, w_start


In [ ]:
index = np.random.randint(len(val_dset))
inp, tar = val_dset[index]
frame, h_start, w_start = get_full_input_frame(index, val_dset)
print(h_start, w_start)

In [ ]:
from disentangle.core.tiff_reader import load_tiff
from disentangle.analysis.paper_plots import show_for_one, get_plotoutput_dir
def get_hwt_start(idx):
    h,w,t = val_dset.idx_manager.hwt_from_idx(idx, grid_size=64)
    print(h,w,t)
    pad = val_dset.per_side_overlap_pixelcount()
    h =  h - pad
    w = w - pad
    return h,w,t

def get_crop_from_fulldset_prediction(full_dset_pred, idx, patch_size=256):
    h,w,t = get_hwt_start(idx)
    return np.swapaxes(full_dset_pred[t,h:h+patch_size,w:w+patch_size].astype(np.float32)[None], 0, 3)[...,0]


In [ ]:
import seaborn as sns
_,ax = plt.subplots(figsize=(12,4),ncols=3)
sns.histplot(np.log(val_dset._data[:,::4,::4,0].reshape(-1,)), ax=ax[0])
sns.histplot(np.log(val_dset._data[:,::4,::4,1].reshape(-1,)), ax=ax[1])
inp = (val_dset._data[...,0] + val_dset._data[...,1])/2
sns.histplot(np.log(inp[:,::4,::4].reshape(-1,)), ax=ax[2])
ax[2].set_title('log(inp)')



In [ ]:
data_mean

In [ ]:
from disentangle.analysis.stitch_prediction import stitch_predictions
from disentangle.analysis.mmse_prediction import get_dset_predictions
# from disentangle.analysis.stitch_prediction import get_predictions as get_dset_predictions

pred_tiled, rec_loss, logvar_tiled, patch_psnr_tuple, pred_std_tiled = get_dset_predictions(model, val_dset,batch_size,
                                               num_workers=num_workers,
                                               mmse_count=mmse_count,
                                                model_type = config.model.model_type,
                                              )
tmp = np.round([x.item() for x in patch_psnr_tuple],2)
print('Patch wise PSNR, as computed during training', tmp,np.mean(tmp))

In [ ]:
pred_tiled.shape

In [ ]:
idx_list = np.where(logvar_tiled.squeeze() < -6)[0]
if len(idx_list) > 0:
    plt.imshow(val_dset[idx_list[0]][1][1])

In [ ]:
len(val_dset)

In [ ]:
if pred_tiled.shape[-1] != val_dset.get_img_sz():
    pad = (val_dset.get_img_sz() - pred_tiled.shape[-1] )//2
    pred_tiled = np.pad(pred_tiled, ((0,0),(0,0),(pad,pad),(pad,pad)))

pred = stitch_predictions(pred_tiled,val_dset, )
if len(np.unique(logvar_tiled)) == 1:
    logvar = None
else:
    logvar = stitch_predictions(logvar_tiled,val_dset)
pred_std = stitch_predictions(pred_std_tiled,val_dset)

In [ ]:
if 'target_idx_list' in config.data and config.data.target_idx_list is not None:
    pred = pred[...,:len(config.data.target_idx_list)]
    pred_std = pred_std[...,:len(config.data.target_idx_list)]

In [ ]:
len(pred) 

In [ ]:
def print_ignored_pixels():
    ignored_pixels = 1
    while(pred[0,-ignored_pixels:,-ignored_pixels:,].std() ==0):
        ignored_pixels+=1
    ignored_pixels-=1
    print(f'In {pred.shape}, last {ignored_pixels} many rows and columns are all zero.')
    return ignored_pixels

actual_ignored_pixels = print_ignored_pixels()

## Ignore the pixels which are present in the last few rows and columns. 
1. They don't come in the batches. So, in prediction, they are simply zeros. So they are being are ignored right now. 
2. For the border pixels which are on the top and the left, overlapping yields worse performance. This is becuase, there is nothing to overlap on one side. So, they are essentially zero padded. This makes the performance worse. 

In [ ]:
actual_ignored_pixels

In [ ]:
ignored_last_pixels = 32 
ignore_first_pixels = 0
assert actual_ignored_pixels <= ignored_last_pixels, f'Set ignored_last_pixels={actual_ignored_pixels}'
print(ignored_last_pixels)

In [ ]:
tar = val_dset._data
if 'target_idx_list' in config.data and config.data.target_idx_list is not None:
    tar = tar[...,config.data.target_idx_list]
def ignore_pixels(arr):
    if ignore_first_pixels:
        arr = arr[:,ignore_first_pixels:,ignore_first_pixels:]
    if ignored_last_pixels:
        arr = arr[:,:-ignored_last_pixels,:-ignored_last_pixels]
    return arr

pred = ignore_pixels(pred)
tar = ignore_pixels(tar)
if pred_std is not None:
    pred_std = ignore_pixels(pred_std)


In [ ]:
sep_mean, sep_std = model.data_mean, model.data_std
if isinstance(sep_mean, dict):
    sep_mean = sep_mean['target']
    sep_std = sep_std['target']

if isinstance(sep_mean, int):
    pass
else:
    sep_mean = sep_mean.squeeze()[None,None,None]
    sep_std = sep_std.squeeze()[None,None,None]
    sep_mean = sep_mean.cpu().numpy() 
    sep_std = sep_std.cpu().numpy()

tar_normalized = (tar - sep_mean)/ sep_std

In [ ]:
from disentangle.metrics.calibration import get_calibrated_factor_for_stdev
from disentangle.analysis.paper_plots import plot_calibration
import numpy as np

def get_plotoutput_dir(ckpt_dir, patch_size, mmse_count=50):
    plotsrootdir = f'/group/jug/ashesh/data/paper_figures/patch_{patch_size}_mmse_{mmse_count}'
    os.makedirs(plotsrootdir, exist_ok=True)
    print(plotsrootdir)
    return plotsrootdir

def get_calibration_fnames(ckpt_dir):
    tokens = ckpt_dir.strip('/').split('/')
    modelid = int(tokens[-1])
    model_specs = tokens[-2].replace('-','')
    monthyear = tokens[-3]
    fname_factor = f'calibration_factor_{monthyear}_{model_specs}_{modelid}.npy'
    fname_stats = f'calibration_stats_{monthyear}_{model_specs}_{modelid}.pkl.npy'
    return {'stats': fname_stats, 'factor': fname_factor}

def get_calibration_factor_fname(ckpt_dir):
    return get_calibration_fnames(ckpt_dir)['factor']

def get_calibration_stats_fname(ckpt_dir):
    return get_calibration_fnames(ckpt_dir)['stats']



In [ ]:
inp = (tar[...,0] - data_mean['target'].squeeze()[0])/data_std['target'].squeeze()[0]
np.quantile(inp, [0, 0.01, 0.1, 0.5,0.9,0.99,1])

In [ ]:
tar.shape

In [ ]:
_,ax = plt.subplots(figsize=(6,4),ncols=2)
ax[0].imshow(tar[0,1000:1500, :500,0])
ax[1].imshow(pred[0,1000:1500, :500,0])

In [ ]:
import matplotlib.patches as patches
import matplotlib
from disentangle.analysis.plot_error_utils import plot_error
nrows = pred.shape[-1]
img_sz = 3
_,ax = plt.subplots(figsize=(4*img_sz,nrows*img_sz),ncols=4,nrows=nrows)
idx = np.random.randint(len(pred))
print(idx)
for ch_id in range(nrows):
  ax[ch_id,0].imshow(tar_normalized[idx,..., ch_id], cmap='magma')
  ax[ch_id,1].imshow(pred[idx,:,:,ch_id], cmap='magma')
  plot_error(tar_normalized[idx,...,ch_id], 
            pred[idx,:,:,ch_id], 
            cmap = matplotlib.cm.coolwarm, 
            ax = ax[ch_id,2], max_val = None)

  cropsz = 256
  h_s = np.random.randint(0, tar_normalized.shape[1] - cropsz)
  h_e = h_s + cropsz
  w_s = np.random.randint(0, tar_normalized.shape[2] - cropsz)
  w_e = w_s + cropsz

  plot_error(tar_normalized[idx,h_s:h_e,w_s:w_e, ch_id], 
            pred[idx,h_s:h_e,w_s:w_e,ch_id], 
            cmap = matplotlib.cm.coolwarm, 
            ax = ax[ch_id,3], max_val = None)

  # Add rectangle to the region
  rect = patches.Rectangle((w_s, h_s), w_e-w_s, h_e-h_s, linewidth=1, edgecolor='r', facecolor='none')
  ax[ch_id,2].add_patch(rect)


In [ ]:
# ch1_pred_unnorm = pred[...,0]*sep_std[...,0].cpu().numpy() + sep_mean[...,0].cpu().numpy()
# ch2_pred_unnorm = pred[...,1]*sep_std[...,1].cpu().numpy() + sep_mean[...,1].cpu().numpy()
pred_unnorm = []
for i in range(pred.shape[-1]):
    if sep_std.shape[-1]==1:
        temp_pred_unnorm = pred[...,i]*sep_std[...,0] + sep_mean[...,0]
    else:
        temp_pred_unnorm = pred[...,i]*sep_std[...,i] + sep_mean[...,i]
    pred_unnorm.append(temp_pred_unnorm)

In [ ]:
import matplotlib.patches as patches

ncols = tar.shape[-1]
imgsz = 4
_,ax = plt.subplots(figsize=((1+ncols)*imgsz,2*imgsz),nrows=2,ncols=ncols + 1)
img_idx = np.random.randint(len(tar))
sz = 900
hs = np.random.randint(tar.shape[1]-sz)
ws = np.random.randint(tar.shape[2]-sz)
print(img_idx, hs, ws)
for i in range(ncols):
    vmin = tar[img_idx,hs:hs+sz, ws:ws+sz ,i].min()
    vmax = tar[img_idx,hs:hs+sz, ws:ws+sz ,i].max()
    ax[0,i+1].imshow(tar[img_idx,hs:hs+sz, ws:ws+sz ,i], vmin=vmin, vmax=vmax)
    ax[1,i+1].imshow(pred_unnorm[i][img_idx,hs:hs+sz, ws:ws+sz], vmin=vmin, vmax=vmax)

if val_dset._input_idx is not None:
    inp = val_dset._data[img_idx,...,val_dset._input_idx]
else:
    inp = np.mean(val_dset._data[img_idx,...], axis=-1)

if val_dset._noise_data is not None:
    inp += val_dset._noise_data[img_idx,...,0]
    
ax[0,0].imshow(inp)
rect = patches.Rectangle((ws, hs), sz,sz, linewidth=2, edgecolor='r', facecolor='none')
ax[0,0].add_patch(rect)

ax[1,0].imshow(inp[hs:hs+sz, ws:ws+sz])
plt.subplots_adjust(wspace=0.03, hspace=0.03)
ax[0,0].set_title('Input')
twinx = ax[0,-1].twinx()
twinx.set_ylabel('Target')
clean_ax(twinx)
twinx = ax[1,-1].twinx()
clean_ax(twinx)
twinx.set_ylabel('Prediction')
clean_ax(ax)


In [ ]:
plt.imshow(pred_unnorm[i][img_idx,hs+120:hs+200, ws:ws+300])

In [ ]:
from stardist.models import StarDist2D
from disentangle.scripts.evaluate import compute_high_snr_stats

model = StarDist2D.from_pretrained('2D_versatile_fluo')



In [ ]:
plt.imshow(pred[0,...,0])

In [ ]:
from tqdm import tqdm
from scipy.spatial.distance import dice

dice_scores = []
for i in tqdm(range(len(pred))):
    label_pred, _ = model.predict_instances(pred[i,...,0])  
    label_tar, _ = model.predict_instances(tar[i,...,0])  
    score = dice(label_pred.reshape(-1,) > 0, label_tar.reshape(-1,) > 0)
    dice_scores.append(score)


In [ ]:
token = f'{DataSplitType.name(eval_datasplit_type)}_{SubDsetType.name(subdset_type)}_P{custom_image_size}_G{image_size_for_grid_centers}_M{mmse_count}_Sk{ignored_last_pixels}'
print(token)
print(f'DICE: {np.mean(dice_scores):.2f} +- {np.std(dice_scores):.4f}')
stats = compute_high_snr_stats(config, tar.astype(np.float32), np.concatenate([x[...,None] for x in pred_unnorm], axis=-1).astype(np.float32))


In [ ]:
# Test_OnlyIba1P50_PNone_G64_M2_Sk32
# DICE: 0.97 +- 0.0012
# PSNR on Highres [34.35, 35.57]
# Multiscale SSIM on Highres [0.982, 0.994]
# Range Invariant Multiscale SSIM on Highres [0.981, 0.994]

In [ ]:
pred.shape

In [ ]:

token = f'{DataSplitType.name(eval_datasplit_type)}_{SubDsetType.name(subdset_type)}_P{custom_image_size}_G{image_size_for_grid_centers}_M{mmse_count}_Sk{ignored_last_pixels}'
fname = token + "_" + '_'.join(ckpt_dir.strip('/').split('/')[-3:]) + '.tif'
fname
 

In [ ]:
data_mean['target']

In [ ]:
import tifffile
fpath = os.path.join('/group/jug/ashesh/naturemethods/iba1ki67', fname)
pred_data = np.concatenate([x[...,None] for x in pred_unnorm], axis=-1)
tifffile.imwrite(fpath, 
pred_data.transpose(0,3,1,2),
imagej=True, 
 metadata={ 'axes': 'ZCYX'}, 
 )
fpath

In [ ]:
fpath_gt = os.path.join('/group/jug/ashesh/naturemethods/iba1ki67', f'GT_{SubDsetType.name(subdset_type)}.tif')
fpath_gt
tifffile.imwrite(fpath_gt, 
tar.transpose(0,3,1,2),
imagej=True, 
 metadata={ 'axes': 'ZCYX'}, 
 )
print(fpath_gt)

In [ ]:
! ls /group/jug/ashesh/naturemethods/iba1ki67/